<a href="https://colab.research.google.com/github/ksushenc1a/-/blob/main/Bert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader

# Загрузка данных
train_data = pd.read_csv('train.csv')  # Замените на актуальный путь
test_data = pd.read_csv('test.csv')    # Замените на актуальный путь

# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Токенизация данных с использованием параллельной обработки
train_encodings = tokenizer(train_data['text'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_data['text'].tolist(), truncation=True, padding=True, return_tensors='pt')

# Преобразование в тензоры
train_labels = torch.tensor(train_data['is_joke'].tolist())
test_labels = torch.tensor(test_data['is_joke'].tolist())

# Создание Dataset
class JokeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}  # Убрали лишнее преобразование в тензор
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JokeDataset(train_encodings, train_labels)
test_dataset = JokeDataset(test_encodings, test_labels)

# Инициализация модели
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Увеличен размер батча
    per_device_eval_batch_size=16,    # Увеличен размер батча
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True,  # Включение смешанной точности
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Обучение модели
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kseniazuravleva (kseniazuravleva-hse) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,0.328900
1000,0.195800


TrainOutput(global_step=1314, training_loss=0.23447869067141272, metrics={'train_runtime': 576.3079, 'train_samples_per_second': 36.418, 'train_steps_per_second': 2.28, 'total_flos': 5522174829895680.0, 'train_loss': 0.23447869067141272, 'epoch': 3.0})

In [ ]:
# Предсказание на тестовых данных
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)

# Создание таблицы с примерами, предсказаниями
results_df = pd.DataFrame({
    'text': test_data['text'],
    'true_label': test_data['is_joke'],
    'predicted_label': predicted_labels
})

# Сохранение результатов в CSV файл
results_df.to_csv('predictions.csv', index=False)


In [ ]:
# Оценка модели
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(test_labels.numpy(), predicted_labels))


              precision    recall  f1-score   support

           0       0.91      0.96      0.93       874
           1       0.96      0.91      0.93       874

    accuracy                           0.93      1748
   macro avg       0.93      0.93      0.93      1748
weighted avg       0.93      0.93      0.93      1748

